In [1]:
import gpytorch
import numpy as np

from bo_functions import best_error_advanced
from bo_functions import calc_ei_advanced
from bo_functions import LHS_Design
from bo_functions import create_y_data
from bo_functions import test_train_split
from bo_functions import ExactGPModel
from bo_functions import train_GP_model
from bo_functions import calc_GP_outputs
from bo_functions import calc_y_expected
from bo_functions import improvement_integral
from bo_functions import improvement_int_terms

from bo_plotters import plotter_adv
from bo_plotters import y_plotter_adv
from bo_plotters import stdev_plotter_adv
from bo_plotters import error_plotter_adv
from bo_plotters import ei_plotter_adv
from bo_plotters import improvement_integral_plot
from bo_plotters import improvement_int_terms_plot
from bo_plotters import plotter_adv_4D
from bo_plotters import y_plotter_adv_4D
from bo_plotters import stdev_plotter_adv_4D
from bo_plotters import error_plotter_adv_4D
from bo_plotters import ei_plotter_adv_4D

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.tri as mtri
import matplotlib

In [2]:
#Set noise parameters and load csv file to generate training data
noise_std = 0.1**2
all_p = LHS_Design("LHS_Toy_3_Input.csv")
all_y = create_y_data(all_p)

#Separate training and testing data, uses default of an 80%/20% split
train_test_data = test_train_split(all_p,all_y)

train_p = train_test_data[0] #1x19
train_y = train_test_data[1] #1x19
test_p = train_test_data[2] #1x19
test_y = train_test_data[3] #1x19
print(train_p)
print(train_y)

tensor([[ 1.8567e+00,  7.6583e-01,  1.2417e+00],
        [-1.5359e+00, -1.4589e+00,  4.0733e-01],
        [ 1.1929e+00, -1.2549e+00, -1.8451e+00],
        [-1.9292e+00,  8.9024e-01,  1.6229e+00],
        [ 2.7640e-01,  6.0036e-02,  1.7340e+00],
        [ 6.7009e-01, -6.7689e-01,  1.1550e+00],
        [-2.6864e-01,  6.1656e-01, -1.7169e+00],
        [ 8.2577e-01,  1.3867e+00, -3.5378e-01],
        [ 1.0064e+00,  3.3627e-01,  6.2007e-01],
        [-1.6244e-03,  1.2003e-01,  7.8550e-01],
        [-1.3450e+00,  4.2636e-01, -8.2567e-01],
        [-9.8063e-01, -1.3159e-01, -4.0761e-01],
        [-1.0562e+00, -1.9284e+00,  3.4419e-01],
        [-1.7425e+00,  1.0433e+00, -1.5221e-01],
        [-1.3687e+00,  1.9121e+00,  2.0146e-01],
        [-1.6021e-01, -7.7287e-01, -6.7441e-01],
        [ 1.2909e+00,  1.7409e+00, -1.3370e+00],
        [-7.1961e-01, -9.3770e-01,  1.4750e+00],
        [-5.2986e-01,  1.5590e+00,  9.2303e-01]], dtype=torch.float64)
tensor([  5.4011,  -0.8001, -12.7549,   3.4883,

In [3]:
# initialize likelihood and model
##Assumes a homoskedastic noise model p(y | f) = f + noise
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# We will use the simplest form of GP model, exact inference
#Defines our model in terms of the class parameters in bo_functions
model = ExactGPModel(train_p, train_y, likelihood)

In [4]:
#Set number of training iterations and train GP
iterations = 500
train_GP_model(model,likelihood, train_p, train_y, iterations)

In [5]:
# Get into evaluation (predictive posterior) mode
#Puts model in evaluation mode
model.eval()
#Puts likelihood in evaluation mode
likelihood.eval();

In [6]:
#Create Meshgrid
point_num = 10  #Any bigger than 10 and the kernel just dies
test_p1 = np.linspace(-2,2,point_num)
test_p2 = np.linspace(-2,2,point_num)
test_p3 = np.linspace(-2,2,point_num)

test_p_mesh = np.array(np.meshgrid(test_p1,test_p2,test_p3))
test_p_mesh_plot = test_p_mesh.T.reshape(-1,3)

In [7]:
# X, Y ,Z = test_p_mesh
# print(X.reshape(-1,4))
# data = z.reshape(point_num,point_num,point_num)

In [8]:
#Calculates GP outputs for mean, variance, standard devaition, and y output
GP_Outputs = calc_GP_outputs(model,likelihood,test_p_mesh_plot)

model_mean = GP_Outputs[0] #1x6
model_variance= GP_Outputs[1] #1x6
model_stdev = GP_Outputs[2] #1x6
model_y = GP_Outputs[3] #1x6

#Calculates expected y
y_exp = calc_y_expected(test_p_mesh_plot) #1x6

# print("Model Mean \n", model_mean)
# print("Model Variance \n", model_variance)
# print("Model Standard Deviation \n", model_stdev)
# print("Model y \n", model_y)
# print("Y Value Expected \n", y_exp)

# print(test_p_mesh_plot)
# print(model_y)

In [9]:
#Calculates best_error and expected improvement
best_error = best_error_advanced(model_y, y_exp)[0] #Scaler
best_x = best_error_advanced(model_y, y_exp)[1] #1x3
# print("best x",best_x)
# print(test_p[1])
ei = calc_ei_advanced(best_error,model_mean,model_variance, y_exp) #1x6

error_mag = abs(y_exp - model_y.numpy())
print("Best Error is: ",best_error)
# print("EI:",ei)
# print(ei.shape)
argmax_ei = np.argmax(ei)
print("argmax is", argmax_ei)
print("The test parameter set that gives the highest expected imrovement is \nTheta1, Theta2, x = \n")
print(test_p_mesh_plot[argmax_ei])

Best Error is:  1.7471285030332852e-05
argmax is 435
The test parameter set that gives the highest expected imrovement is 
Theta1, Theta2, x = 

[-0.66666667  0.22222222 -0.22222222]


## Analysis of Expected Improvement
 - Expected Improvement is largest farther from the edges
  - This is rational because you can't explore any further than the edges
 - Expected Improvement increases as error decreases
  - This is rational because as error decreases, more exploitation is possible
 - This means we are most likely to sample in the middle, farthet from the edges


## Analysis of Standard Deviation
 - The GP estimates that the standard deviation is lowest at points that were directly tested
  - This can be rationalized by the way that the contour plot is drawn
 - Standard deviation is smallest away from the edges and larger towards them
  - This is rationalized by the fact that there are less neighbors that the GP is tested and trained with at the boundaries
 - The more points that get tested, the more the standard deviations will decrease

## Analysis of Error Magnitude
 - The GP emulator is most inaccurate when all values of $\bar{p}$ are at their maximum. 
  - In general, the GP is less accurate at extreme points, this is rationalized by the fact that there are less neighbors that the GP is tested and trained with at the boundaries
 - The GP emulator is most accurate when x is at it's maximum, but $\bar{\Theta}= 0$
  - This is rationalized by the fact that multiple terms become zero if any of the values of $\bar{p}$ are zero 
 - GP error is mostly very high, as more iterations are added, these will decrease

## Analysis of GP Emulator (Model y)
 - The GP emulator correctly captures that y increases as $\bar{p}$ increases. This tells us that this GP emulator model could be viable
  - The GP emulator correctly estimates where the lowest y is achieved, but not the actual value of y
  - The GP emulator slightly mistakes where the most positive value of y is, and does not predict the actual value of y
 - The model as it is is inaccurate, BO should increase the accuracy of the emulator

In [10]:
# print(ei_plotter_adv(test_p_mesh_plot, ei))